In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib

# Load dataset
df = pd.read_csv(r"D:\\Codes\\Projects\\ML\\Limi\data\\RTA Dataset.csv")

# Drop rows with missing target
df.dropna(subset=['Accident_severity'], inplace=True)

# Encode target
severity_order = ['Slight Injury', 'Serious Injury', 'Fatal injury']
df['Accident_severity'] = pd.Categorical(df['Accident_severity'], categories=severity_order, ordered=True)
le = LabelEncoder()
df['Severity_encoded'] = le.fit_transform(df['Accident_severity'])

# Fill missing values
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    df[col] = df[col].fillna(df[col].mean())

# Drop columns
df.drop(columns=['Accident_severity', 'Time'], inplace=True, errors='ignore')

# One-hot encode categorical features
df = pd.get_dummies(df, drop_first=True)

# Features and labels
X = df.drop(columns=['Severity_encoded'])
y = df['Severity_encoded']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network
nn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])
nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
nn.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=0)

# Evaluate
y_pred_nn = nn.predict(X_test_scaled).argmax(axis=1)
print("MSE:", mean_squared_error(y_test, y_pred_nn))
print("MAE:", mean_absolute_error(y_test, y_pred_nn))

# Save model and scaler
nn.save('neural_network_model.h5')
joblib.dump(scaler, 'scaler.pkl')


d:\Codes\Projects\ML\Limi\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


MSE: 0.23286781422539785
MAE: 0.20428710620331275


['scaler.pkl']